In [ ]:
import pandas as pd
import scipy.stats as sps
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [ ]:
#for training

atp_matches_2024 = pd.read_csv('atp_matches_2024.csv')
atp_matches_2023 = pd.read_csv('atp_matches_2023.csv')
atp_matches_2022 = pd.read_csv('atp_matches_2022.csv')
atp_matches_qual_chall_2023 = pd.read_csv('atp_matches_qual_chall_2023.csv')
atp_matches_qual_chall_2024 = pd.read_csv('atp_matches_qual_chall_2024.csv')
atp_matches_qual_chall_2022 = pd.read_csv('atp_matches_qual_chall_2022.csv')
all_data = pd.concat([atp_matches_2024,atp_matches_2023, atp_matches_2022, atp_matches_qual_chall_2024, atp_matches_qual_chall_2023, atp_matches_qual_chall_2022], ignore_index=True)

all_data = all_data.dropna(subset=['w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon'])

all_data['tourney_name'] = all_data['tourney_name'].apply(lambda x: 'Davis Cup' if x.startswith('Davis Cup') else x)
all_data = all_data[all_data['tourney_name']!='Davis Cup']
all_data = all_data[all_data['tourney_name']!='United Cup']
all_data = all_data[all_data['surface'].isin(['Hard','Clay','Grass'])]
all_data = all_data[all_data['best_of'] == 3]
all_data

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2024-0339,Brisbane,Hard,32,A,20240101,300,105777,2,NaN,...,58.0,44.0,16.0,11.0,8.0,9.0,14.0,2570.0,8.0,3660.0
1,2024-0339,Brisbane,Hard,32,A,20240101,299,208029,1,NaN,...,35.0,31.0,10.0,11.0,5.0,7.0,8.0,3660.0,39.0,1122.0
2,2024-0339,Brisbane,Hard,32,A,20240101,298,105777,2,NaN,...,39.0,24.0,14.0,10.0,5.0,7.0,14.0,2570.0,55.0,902.0
3,2024-0339,Brisbane,Hard,32,A,20240101,297,208029,1,NaN,...,51.0,31.0,16.0,10.0,3.0,5.0,8.0,3660.0,116.0,573.0
4,2024-0339,Brisbane,Hard,32,A,20240101,296,126128,NaN,NaN,...,37.0,27.0,16.0,10.0,5.0,8.0,39.0,1122.0,44.0,1021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32468,2022-0352,Paris Masters,Hard,64,M,20221031,190,106329,13.0,NaN,...,48.0,39.0,17.0,15.0,2.0,4.0,66.0,770.0,118.0,488.0
32469,2022-0352,Paris Masters,Hard,64,M,20221031,189,209950,NaN,WC,...,32.0,22.0,9.0,9.0,3.0,7.0,308.0,164.0,56.0,839.0
32470,2022-0352,Paris Masters,Hard,64,M,20221031,188,104926,8.0,NaN,...,61.0,43.0,11.0,11.0,4.0,6.0,59.0,792.0,68.0,750.0
32471,2022-0352,Paris Masters,Hard,64,M,20221031,187,208103,NaN,NaN,...,34.0,26.0,11.0,10.0,2.0,4.0,75.0,707.0,52.0,891.0


In [ ]:
all_data['surface'].value_counts()

surface
Hard     15013
Clay     14281
Grass     1060
Name: count, dtype: int64

In [ ]:
all_data['best_of'].value_counts()
#this info before deleting matches which are 'best of 5'

best_of
3    30769
5     1184
Name: count, dtype: int64

In [ ]:
def players_statistics(data):
  names = set(list(all_data['winner_name'].unique()) + list(all_data['loser_name'].unique()))
  names = list(names)
  players_stat = {name: [0,0,0,0,0,0,0,0,0,0,0,0,0] for name in names}
  for name in names:
    players_stat[name][0] = data.loc[data['winner_name'] == name]['w_svpt'].sum() + data.loc[data['loser_name'] == name]['l_svpt'].sum()
    players_stat[name][1] = data.loc[data['winner_name'] == name]['w_1stIn'].sum() + data.loc[data['loser_name'] == name]['l_1stIn'].sum()
    players_stat[name][2] = data.loc[data['winner_name'] == name]['w_1stWon'].sum() + data.loc[data['loser_name'] == name]['l_1stWon'].sum()
    players_stat[name][3] = data.loc[data['winner_name'] == name]['w_2ndWon'].sum() + data.loc[data['loser_name'] == name]['l_2ndWon'].sum()

    players_stat[name][4] = data.loc[data['winner_name'] == name]['l_svpt'].sum() + data.loc[data['loser_name'] == name]['w_svpt'].sum()
    players_stat[name][5] = data.loc[data['winner_name'] == name]['l_1stIn'].sum() + data.loc[data['loser_name'] == name]['w_1stIn'].sum()
    players_stat[name][6] = data.loc[data['winner_name'] == name]['l_1stWon'].sum() + data.loc[data['loser_name'] == name]['w_1stWon'].sum()
    players_stat[name][7] = data.loc[data['winner_name'] == name]['l_2ndWon'].sum() + data.loc[data['loser_name'] == name]['w_2ndWon'].sum()

    if(players_stat[name][0] == 0 or players_stat[name][1] == 0 or players_stat[name][4] == 0 or players_stat[name][5] == 0):
      players_stat.pop(name)
      continue

    players_stat[name][8] = players_stat[name][1]/players_stat[name][0] #a_i

    players_stat[name][9] = players_stat[name][2]/players_stat[name][1] #b_i

    players_stat[name][10] = players_stat[name][3]/(players_stat[name][0] - players_stat[name][1]) #c_i

    players_stat[name][11] = (players_stat[name][5] - players_stat[name][6])/(players_stat[name][5]) #d_i

    players_stat[name][12] = (players_stat[name][4] - players_stat[name][5] - players_stat[name][7])/(players_stat[name][4] - players_stat[name][5]) #e_i


  df = pd.DataFrame(players_stat)

  return df

In [ ]:
def prob_calc(players_stats, player1):
  a_av = players_stats.iloc[8].mean()
  f_i = players_stats.loc[8][player1]*players_stats.loc[9][player1] + (1- players_stats.loc[8][player1])*players_stats.loc[10][player1]
  g_i = a_av*players_stats.loc[11][player1] + (1-a_av)*players_stats.loc[12][player1]

  return f_i,g_i

In [ ]:
#player1 serves
#function calculate prob of winning a point by player1 and prob of winning a point by player2(who returns)
def combined_prob_calc(players_stats, games_stats,tournament_name, player1, player2):
  f_av = 0
  g_av = 0
  l = 0
  for name in players_stats.columns:
    f_av += prob_calc(players_stats, name)[0]
    g_av += prob_calc(players_stats, name)[1]
    l+=1
  f_av = f_av/l
  g_av = g_av/l
  games_stats = games_stats.loc[games_stats['tourney_name'] == tournament_name]
  players_stats_tournament = players_statistics(games_stats)
  l = 0
  f_t = 0
  g_t = 0
  for name in players_stats_tournament.columns:
    f_t += prob_calc(players_stats_tournament, name)[0]
    g_t += prob_calc(players_stats_tournament, name)[1]
    l+=1
  f_t = f_t/l
  g_t = g_t/l

  f_i, g_i = prob_calc(players_stats, player1)
  f_j, g_j = prob_calc(players_stats, player2)

  f_ij = f_t + (f_i - f_av) - (g_j - g_av)
  g_ji = g_t + (g_j - g_av) - (f_i - f_av)
  return f_ij, g_ji

In [ ]:
#player A is surving first during a game
def game_prob_calc(p, x, y):
  if(x == 4 and x-y>=2):
    return 1
  elif(y == 4 and y-x>=2):
    return 0
  elif(x == 3 and y == 3):
    return (p*p)/(p*p+(1-p)*(1-p))
  else:
    return p*game_prob_calc(p, x+1, y) + (1-p)*game_prob_calc(p, x, y+1)

In [ ]:
#player A is surving first
def tiebreaker_prob_calc(p_a,p_b,x,y):
  if(x==7 and x-y>=2):
    return 1
  elif(y==7 and y-x>=2):
    return 0
  elif(x==6 and y==6):
    return (p_a*(1-p_b))/(p_a*(1-p_b)+(1-p_a)*p_b)
  elif((x+y+3)%4>=0 and (x+y+3)%4<=1):
    return p_b*tiebreaker_prob_calc(p_a,p_b,x,y+1) + (1-p_b)*tiebreaker_prob_calc(p_a,p_b,x+1,y)
  elif((x+y+3)%4>=2 and (x+y+3)%4<=3):
    return p_a*tiebreaker_prob_calc(p_a,p_b,x+1,y) + (1-p_a)*tiebreaker_prob_calc(p_a,p_b,x,y+1)

In [ ]:
#player A serves first in first game of this set
def set_prob_calc(p_a,p_b,x,y):
  p_a_game = game_prob_calc(p_a, 0, 0)
  p_b_game = game_prob_calc(p_b, 0, 0)
  if(x>=6 and x-y>=2):
    return 1
  elif(y>=6 and y-x>=2):
    return 0
  elif(y==6 and x == 6):
    return tiebreaker_prob_calc(p_a,p_b,0,0)
  elif((x+y)%2 == 0):
    return p_a_game*set_prob_calc(p_a,p_b,x+1,y) + (1-p_a_game)*set_prob_calc(p_a,p_b,x,y+1)
  elif((x+y)%2 == 1):
    return p_b_game*set_prob_calc(p_a,p_b,x,y+1) + (1-p_b_game)*set_prob_calc(p_a,p_b,x+1,y)

In [ ]:
#it assumes that player A serves first in first set, second player serves in second set and so on..
def match_prob_calc(p_a, p_b, x, y, best_of):
  p_a_set = set_prob_calc(p_a,p_b,0,0)
  p_b_set = set_prob_calc(p_b,p_a,0,0)
  if(best_of == 3):
    if(x==2 and y<2):
      return 1
    elif(y==2 and x<2):
      return 0
    elif(x==1 and y==1):
      return p_a_set

  if(best_of == 5):
    if(x==3 and y<3):
      return 1
    elif(y==3 and x<3):
      return 0
    elif(x==2 and y==2):
      return p_a_set

  if((x+y)%2 == 0):
    return p_a_set*match_prob_calc(p_a,p_b,x+1,y, best_of) + (1-p_a_set)*match_prob_calc(p_a,p_b,x,y+1,best_of)
  elif((x+y)%2==1):
    return p_b_set*match_prob_calc(p_a,p_b,x,y+1, best_of) + (1-p_b_set)*match_prob_calc(p_a,p_b,x+1,y, best_of)

In [ ]:
#for realization with common opponents
def match_odds(player1, player2,games_states, best_of):
  player1_opponents = set(list(games_states[games_states['winner_name'] == player1]['loser_name']) + list(games_states[games_states['loser_name'] == player1]['winner_name']))
  player2_opponents = set(list(games_states[games_states['winner_name'] == player2]['loser_name']) + list(games_states[games_states['loser_name'] == player2]['winner_name']))
  mutual_opponents = list(player1_opponents & player2_opponents)
  if(len(mutual_opponents) <= 1):
    return 'no mutual opponents'
  p_avg = 0
  print(mutual_opponents)
  for C_i in mutual_opponents:
    spw_A_Ci = 0
    rpw_A_Ci = 0
    spw_B_Ci = 0
    rpw_B_Ci = 0
    temp1 = games_states[(games_states['winner_name']==player1) & (games_states['loser_name']==C_i)]
    temp2 = games_states[(games_states['winner_name']==C_i) & (games_states['loser_name']==player1)]
    spw_A_Ci += (temp1['w_1stWon'].sum() + temp1['w_2ndWon'].sum() + temp2['l_1stWon'].sum() + temp2['l_2ndWon'].sum())
    spw_A_Ci /= (temp1['w_svpt'].sum() + temp2['l_svpt'].sum())
    rpw_A_Ci += (temp1['l_1stWon'].sum() + temp1['l_2ndWon'].sum() + temp2['w_1stWon'].sum() + temp2['w_2ndWon'].sum())
    rpw_A_Ci /= (temp1['l_svpt'].sum() + temp2['w_svpt'].sum())
    rpw_A_Ci = 1 - rpw_A_Ci

    temp1 = games_states[(games_states['winner_name']==player2) & (games_states['loser_name']==C_i)]
    temp2 = games_states[(games_states['winner_name']==C_i) & (games_states['loser_name']==player2)]
    spw_B_Ci += (temp1['w_1stWon'].sum() + temp1['w_2ndWon'].sum() + temp2['l_1stWon'].sum() + temp2['l_2ndWon'].sum())
    spw_B_Ci /= (temp1['w_svpt'].sum() + temp2['l_svpt'].sum())
    rpw_B_Ci += (temp1['l_1stWon'].sum() + temp1['l_2ndWon'].sum() + temp2['w_1stWon'].sum() + temp2['w_2ndWon'].sum())
    rpw_B_Ci /= (temp1['l_svpt'].sum() + temp2['w_svpt'].sum())
    rpw_B_Ci = 1 - rpw_B_Ci

    delta_i = (spw_A_Ci - (1-rpw_A_Ci)) - (spw_B_Ci-(1-rpw_B_Ci))
    print(delta_i)
    prob = (match_prob_calc(0.6+delta_i, 0.4, 0, 0, best_of) + match_prob_calc(0.6, 0.4 + delta_i, 0, 0, best_of))/2
    p_avg += prob
  p_avg /=len(mutual_opponents)
  return p_avg

In [ ]:
new_data = players_statistics(all_data)

In [ ]:
#test_data = all_data.loc[all_data['tourney_date']>=20240520]
test_data = pd.read_excel('odds.xlsx')
train_data = all_data.loc[all_data['tourney_date']<20240520]
test_data

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,...,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,winner_odds,loser_odds
0,1248,2024-0322,Geneva,Clay,32,A,20240520,300,134770,2.0,...,18,10,1,4,7,4185,44.0,1029.0,1.62,2.30
1,1249,2024-0322,Geneva,Clay,32,A,20240520,299,207830,NaN,...,11,12,7,12,44,1029,1.0,9860.0,5.00,1.17
2,1250,2024-0322,Geneva,Clay,32,A,20240520,298,134770,2.0,...,16,13,6,9,7,4185,56.0,860.0,1.20,4.50
3,1251,2024-0322,Geneva,Clay,32,A,20240520,297,104925,1.0,...,9,10,5,9,1,9860,27.0,1605.0,1.13,6.00
4,1252,2024-0322,Geneva,Clay,32,A,20240520,296,207830,NaN,...,6,10,1,3,44,1029,65.0,795.0,1.44,2.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,8920,2024-2869,Skopje CH,Clay,32,C,20240520,242,207764,NaN,...,12,9,6,10,428,108,1705.0,1.0,1.02,13.00
209,8922,2024-2869,Skopje CH,Clay,32,C,20240520,240,208843,5.0,...,8,9,2,7,470,92,NaN,NaN,1.04,10.00
210,8923,2024-2869,Skopje CH,Clay,32,C,20240520,239,207484,12.0,...,15,10,3,6,636,53,955.0,15.0,1.30,3.40
211,8924,2024-2869,Skopje CH,Clay,32,C,20240520,238,144970,6.0,...,4,6,0,5,487,87,NaN,NaN,1.01,17.00


In [ ]:
#for method №3 (see lower)
train_players_data_HARD_bestof3 = players_statistics(train_data[(train_data['surface'] == 'Hard') & (train_data['best_of'] == 3)])
train_players_data_HARD_bestof5 = players_statistics(train_data[(train_data['surface'] == 'Hard') & (train_data['best_of'] == 5)])
train_players_data_CLAY_bestof3 = players_statistics(train_data[(train_data['surface'] == 'Clay') & (train_data['best_of'] == 3)])
train_players_data_CLAY_bestof5 = players_statistics(train_data[(train_data['surface'] == 'Clay') & (train_data['best_of'] == 5)])
train_players_data_GRASS_bestof3 = players_statistics(train_data[(train_data['surface'] == 'Grass') & (train_data['best_of'] == 3)])
train_players_data_GRASS_bestof5 = players_statistics(train_data[(train_data['surface'] == 'Grass') & (train_data['best_of'] == 5)])
train_players_data_CARPET_bestof3 = players_statistics(train_data[(train_data['surface'] == 'Carpet') & (train_data['best_of'] == 3)])
train_players_data_CARPET_bestof5 = players_statistics(train_data[(train_data['surface'] == 'Carpet') & (train_data['best_of'] == 5)])
#не актуально т.к я убрал игры с пятью сетами

In [ ]:
#for method №1
train_players_data = players_statistics(train_data)

In [ ]:
###### for method №2
train_players_data_HARD = players_statistics(train_data[train_data['surface'] == 'Hard'])
train_players_data_CLAY = players_statistics(train_data[train_data['surface'] == 'Clay'])
train_players_data_GRASS = players_statistics(train_data[train_data['surface'] == 'Grass'])
#train_players_data_CARPET = players_statistics(train_data[train_data['surface'] == 'Carpet'])


In [ ]:
def bet(row, train_players_data, train_data, first_serve, best_of, bet_sum, revenue):
  if((row['winner_name'] in train_players_data.columns) and (row['loser_name'] in train_players_data.columns) and row['tourney_name'] in list(train_data['tourney_name'])):
      f_ij = combined_prob_calc(train_players_data, train_data, row['tourney_name'], row['winner_name'], row['loser_name'])[0] #prob of first player to win on his serve
      g_ji = combined_prob_calc(train_players_data, train_data, row['tourney_name'], row['loser_name'], row['winner_name'])[0] #prob of second player to win on his serve
      print(f_ij, g_ji)
      if(first_serve == 0):
        A = match_prob_calc(f_ij, g_ji, 0, 0, best_of)
        if(row['winner_odds']>1/A):
          bet_sum+=1
          revenue+=row['winner_odds']
        elif(row['loser_odds']>1/(1-A)):
          bet_sum+=1
        print(row['winner_name'], ': ', 1/A,'    ', row['loser_name'], ': ', 1/(1-A))
      else:
        A = match_prob_calc(g_ji, f_ij, 0, 0, best_of)
        if(row['winner_odds']>1/(1-A)):
          bet_sum+=1
          revenue+=row['winner_odds']
        elif(row['loser_odds']>1/A):
          bet_sum+=1
        print(row['loser_name'], ': ', 1/A,'    ', row['winner_name'], ': ', 1/(1-A))
        return [bet_sum, revenue]

In [ ]:
#1 basic implementation of Markov model
revenue = 0
bet_sum = 0
for ind, row in test_data.iterrows():
  first_serve = sps.bernoulli.rvs(0.5)
  best_of = row['best_of']
  if((row['winner_name'] in train_players_data.columns) and (row['loser_name'] in train_players_data.columns) and row['tourney_name'] in list(train_data['tourney_name'])):
    f_ij = combined_prob_calc(train_players_data, train_data, row['tourney_name'], row['winner_name'], row['loser_name'])[0] #prob of first player to win on his serve
    g_ji = combined_prob_calc(train_players_data, train_data, row['tourney_name'], row['loser_name'], row['winner_name'])[0] #prob of second player to win on his serve
    print(f_ij, g_ji) #print probs
    if(first_serve == 0):
      A = match_prob_calc(f_ij, g_ji, 0, 0, best_of)
      #we make a bet only if calculated odd is lower than bookmakers odd
      if(row['winner_odds']>1/A):
        bet_sum+=1
        revenue+=row['winner_odds']
      elif(row['loser_odds']>1/(1-A)):
        bet_sum+=1
      print(row['winner_name'], ': ', 1/A,'    ', row['loser_name'], ': ', 1/(1-A)) #print calculated odds
    else:
      A = match_prob_calc(g_ji, f_ij, 0, 0, best_of)
      if(row['winner_odds']>1/(1-A)):
        bet_sum+=1
        revenue+=row['winner_odds']
      elif(row['loser_odds']>1/A):
        bet_sum+=1
      print(row['loser_name'], ': ', 1/A,'    ', row['winner_name'], ': ', 1/(1-A))

print('revenue: ', revenue)
print('bet_sum: ', bet_sum)

NameError: name 'train_players_data' is not defined

In [ ]:
#2 here we take into account type of surface
revenue = 0
bet_sum = 0
for ind, row in test_data.iterrows():
  best_of = row['best_of']
  if(row['surface'] == 'Hard'):
    train_players_data = train_players_data_HARD
  elif(row['surface'] == 'Clay'):
    train_players_data = train_players_data_CLAY
  elif(row['surface'] == 'Grass'):
    train_players_data = train_players_data_GRASS
  elif(row['surface'] == 'Carpet'):
    train_players_data = train_players_data_CARPET

  if((row['winner_name'] in train_players_data.columns) and (row['loser_name'] in train_players_data.columns) and row['tourney_name'] in list(train_data['tourney_name'])):
    f_ij = combined_prob_calc(train_players_data, train_data, row['tourney_name'], row['winner_name'], row['loser_name'])[0] #prob of first player to win on his serve
    g_ji = combined_prob_calc(train_players_data, train_data, row['tourney_name'], row['loser_name'], row['winner_name'])[0] #prob of second player to win on his serve
    print(f_ij, g_ji) #print probabilities
    A = (match_prob_calc(f_ij, g_ji, 0, 0, best_of) + 1 - match_prob_calc(g_ji, f_ij, 0, 0, best_of))/2
    if(row['winner_odds']>1/A):
      bet_sum+=1
      revenue+=row['winner_odds']
    elif(row['loser_odds']>1/(1-A)):
      bet_sum+=1
    print(row['winner_name'], ': ', 1/A,'    ', row['loser_name'], ': ', 1/(1-A)) #print calculated odds

print('revenue: ', revenue)
print('bet_sum: ', bet_sum)

0.6597342696360573 0.6337715114340677
Casper Ruud :  1.5970421678592348      Tomas Machac :  2.6749235712874655
0.6078201683990854 0.6375191808140207
Tomas Machac :  2.831371863290445      Novak Djokovic :  1.5460387483529914
0.6669740270169504 0.6202475779874828
Casper Ruud :  1.3903406578547965      Flavio Cobolli :  3.561864822116459
0.6789350257032736 0.6408231798002583
Novak Djokovic :  1.4722426307344907      Tallon Griekspoor :  3.11755554225309
0.6438995592955332 0.5847585824577268
Tomas Machac :  1.2904391190388642      Alex Michelsen :  4.443062364702285
0.598281675965201 0.6015660648917465
Flavio Cobolli :  2.07014400563752      Alexander Shevchenko :  1.9344536760772368
0.6603188707152648 0.6417148499547627
Casper Ruud :  1.6927289700059238      Sebastian Baez :  2.443566016867244
0.654396212485028 0.6192789441888461
Novak Djokovic :  1.4929068861333237      Yannick Hanfmann :  3.0287807456792466
0.7280280485557777 0.6612903694745467
Tallon Griekspoor :  1.2769747207394448 

In [ ]:
# i forgot to save my probabilities and odds from previous cell into dataframe, so use it as a text:
text = '''0.6597342696360573 0.6337715114340677
Casper Ruud :  1.5970421678592348      Tomas Machac :  2.6749235712874655
0.6078201683990854 0.6375191808140207
Tomas Machac :  2.831371863290445      Novak Djokovic :  1.5460387483529914
0.6669740270169504 0.6202475779874828
Casper Ruud :  1.3903406578547965      Flavio Cobolli :  3.561864822116459
0.6789350257032736 0.6408231798002583
Novak Djokovic :  1.4722426307344907      Tallon Griekspoor :  3.11755554225309
0.6438995592955332 0.5847585824577268
Tomas Machac :  1.2904391190388642      Alex Michelsen :  4.443062364702285
0.598281675965201 0.6015660648917465
Flavio Cobolli :  2.07014400563752      Alexander Shevchenko :  1.9344536760772368
0.6603188707152648 0.6417148499547627
Casper Ruud :  1.6927289700059238      Sebastian Baez :  2.443566016867244
0.654396212485028 0.6192789441888461
Novak Djokovic :  1.4929068861333237      Yannick Hanfmann :  3.0287807456792466
0.7280280485557777 0.6612903694745467
Tallon Griekspoor :  1.2769747207394448      Denis Shapovalov :  4.610437794937676
0.6238006547200883 0.7092207434075857
Alex Michelsen :  6.646213633103876      Taylor Fritz :  1.177109841210573
0.6356775895606445 0.6338967325182844
Tomas Machac :  1.964980232464765      Nicolas Moreno De Alboran :  2.036290657939994
0.6723474210601298 0.6573411775000604
Flavio Cobolli :  1.747180450838908      Ben Shelton :  2.3383647803917182
0.6347589046041826 0.6166733714813973
Sebastian Baez :  1.6935028599758655      Roberto Carballes Baena :  2.441955120465979
0.6873280895381876 0.6540580546177224
Casper Ruud :  1.5243878538561895      Sebastian Ofner :  2.906985435772974
0.6498464578937201 0.6386098568430767
Yannick Hanfmann :  1.8001253971248004      Andy Murray :  2.249804097699481
0.6134973546711472 0.679098036766717
Denis Shapovalov :  4.8014152299127995      Federico Coria :  1.263059923612433
0.744073931747128 0.6538561914940046
Tallon Griekspoor :  1.171501858745317      Christopher Eubanks :  6.83084059447435
0.6838560068912933 0.6600658869569964
Alex Michelsen :  1.6317054208687387      Aleksandar Kovacevic :  2.583016334773212
0.6645430480816928 0.6468736218493355
Nicolas Moreno De Alboran :  1.7069160538460952      David Goffin :  2.414595119971222
0.6612051213328525 0.6416002930719286
Tomas Machac :  1.6792236017476567      Jack Draper :  2.472269216539264
0.6041314216815578 0.6442036784656378
Alexander Shevchenko :  3.2690723537474766      Fabian Marozsan :  1.4407087320721412
0.6538694947497814 0.6162066246836979
Emil Ruusuvuori :  1.4671788809058264      Marcos Giron :  3.140507717431644
0.6328509101892397 0.6277102725748196
Flavio Cobolli :  1.9017954739152787      Aslan Karatsev :  2.108898889964874
0.6070924008059917 0.5898951614583113
Sebastian Baez :  1.7004042785395184      Sumit Nagal :  2.4277468465572456
0.6278033679061228 0.6340740018410662
Roberto Carballes Baena :  2.1343061962727217      Daniel Altmaier :  1.8815961715504634
0.7051894547423656 0.6321664246192169
Sebastian Ofner :  1.232640361516126      Rinky Hijikata :  5.29848025288029
0.6734619183505983 0.7181928102924815
Giovanni Mpetshi Perricard :  3.3320920233404006      Tomas Martin Etcheverry :  1.428799545640415
0.6158156783955124 0.6159234283618376
Tomas Martin Etcheverry :  2.0021973859213205      Luciano Darderi :  1.9978074319967414
0.749080626375134 0.6850071158988036
Giovanni Mpetshi Perricard :  1.3026986985150322      Alexander Bublik :  4.303615129188735
0.6221902957463663 0.6201745467578925
Tomas Martin Etcheverry :  1.9599532155348545      Dominik Koepfer :  2.041717433534334
0.6640099293561113 0.6748368564664229
Giovanni Mpetshi Perricard :  2.2325061328544558      Hugo Gaston :  1.8113549891098903
0.6028072225974345 0.6908294060616273
Alexander Bublik :  7.237228509932459      Pavel Kotov :  1.160327619616237
0.6211393015467703 0.6082957758591294
Dominik Koepfer :  1.7693530242705773      Ugo Humbert :  2.299793421814516
0.6236035574165096 0.623732745227958
Tomas Martin Etcheverry :  2.002616790258427      Thiago Seyboth Wild :  1.9973900394608863
0.6433736246473511 0.513250806435761
Luciano Darderi :  1.0466987544694017      Adrian Mannarino :  22.41384735764688
0.6898081856572353 0.6800978267077411
Arthur Rinderknech :  1.8318453420900755      Frances Tiafoe :  2.20214654022997
0.5734373804867422 0.5922640099929906
Hugo Gaston :  2.4834123681789477      Francisco Cerundolo :  1.6741213848901708
0.6157380040840121 0.6243028030254628
Pavel Kotov :  2.1899654297015583      Alexandre Muller :  1.840360547491534
0.6230146239905174 0.6864172446977428
Alexander Bublik :  4.583360424857933      Aleksandar Vukic :  1.2790676575716344
0.6111091758475597 0.5829162345238363
Dominik Koepfer :  1.5562325987728025      Javier Barranco Cosano :  2.7978090500381794
0.611620004829519 0.5860766142867382
Thiago Seyboth Wild :  1.5885162963091892      Pedro Martinez :  2.6991882914226544
0.6198260877110358 0.5705645558142396
Tomas Martin Etcheverry :  1.3539816339180943      Nikolas Sanchez Izquierdo :  3.8250053228224368
0.6080484807836042 0.5892657380294395
Luciano Darderi :  1.6777061667312891      Taro Daniel :  2.4755657379114573
0.6918256894542464 0.6549811228311428
Arthur Rinderknech :  1.4897956341738507      Daniel Evans :  3.0416678513003133
0.6073309548169851 0.6416925622073395
Frances Tiafoe :  3.0143808649247266      Jaume Munar :  1.4964304503743229
0.5899923445011117 0.6147516610833277
Yoshihito Nishioka :  2.6691595824898156      Sebastian Fanselow :  1.5991038906587602
0.7194521038946199 0.7192862348062424
Giovanni Mpetshi Perricard :  1.996991994710627      Lorenzo Sonego :  2.003017080684029
0.5962407162728477 0.6141239217493205
Hugo Gaston :  2.4454599464475315      Daniel Elahi Galan :  1.6918213143558032
0.6189300531868408 0.6367888434407858
Pavel Kotov :  2.434280022392331      Pedro Cachin :  1.697213922238165
0.6257945160186122 0.6001152756894097
Alexandre Muller :  1.5904321534272754      Richard Gasquet :  2.6936746994473633
0.6284649575214616 0.5831457971875295
Aleksandar Vukic :  1.3887418013635002      Mackenzie Mcdonald :  3.572401518160717
0.672206541714212 0.6417584185564448
Sebastian Ofner :  1.5492134788778815      Ivan Gakhov :  2.820785611531489
0.6614957616549287 0.6745571442407579
Aleksandar Kovacevic :  2.287470669486866      Abedallah Shelbayh :  1.7767167234951922
0.6470956248330233 0.6512579922409079
David Goffin :  2.085582218510347      Gijs Brouwer :  1.9211646828300257
0.6628281963363797 0.6370868511889215
Nicolas Moreno De Alboran :  1.6007343893497974      Juan Pablo Ficovich :  2.664629190085732
0.7214238831819995 0.6325194933061299
Sebastian Ofner :  1.1677970276308218      Hendrik Jebens :  6.959581132748953
0.6510569532151819 0.6348171436968317
Ivan Gakhov :  1.7239727748798543      Ethan Quinn :  2.381267410457463
0.6496752772220379 0.7198097881917362
Aleksandar Kovacevic :  4.934924645083814      Mitchell Krueger :  1.254134472752705
0.5716019653667859 0.6180475689807299
Abedallah Shelbayh :  3.6578671007572923      Damien Wenger :  1.3762415358221165
0.6577890928837852 0.6514415198493565
David Goffin :  1.8829175180510096      Gianluca Mager :  2.132608629407924
0.7256347370949324 0.681224762789868
Gijs Brouwer :  1.4349249843773706      Antoine Bellier :  3.299247079198218
0.7098054677244166 0.6194407542845273
Nicolas Moreno De Alboran :  1.1580105440807813      Thai Son Kwiatkowski :  7.328691580789447
0.605769590220088 0.5903442818590866
Juan Pablo Ficovich :  1.7266008227407372      Jakub Paul :  2.376271494199527
0.621716175605146 0.5678845443712536
Taro Daniel :  1.3201268900793222      Sebastian Fanselow :  4.1237613302406935
0.5614991849039067 0.5805520264885058
Nikolas Sanchez Izquierdo :  2.4947940911440396      Javier Barranco Cosano :  1.6689884619724784
0.6320911225698439 0.6209873714101006
Daniel Elahi Galan :  1.7991421174516737      Harold Mayot :  2.2513418804515366
0.588444738761549 0.6194252748930449
Hugo Gaston :  2.89999851132127      Pedro Cachin :  1.526316201850387
0.655682552214142 0.5614323875137291
Taro Daniel :  1.1282044761467327      Maxime Chazal :  8.800039671434556
0.5994742264767423 0.6370695109664337
Sebastian Fanselow :  3.163835590213175      Thiago Agustin Tirante :  1.4621423201110595
0.5985351535325332 0.5535872486748126
Nikolas Sanchez Izquierdo :  1.3839221782408648      Constant Lestienne :  3.6046945362260905
0.6299015319841886 0.5997250798235537
Javier Barranco Cosano :  1.5383320883043377      Pierre Hugues Herbert :  2.857589435453949
0.661135143087472 0.6277689080579252
Daniel Elahi Galan :  1.5135566991982194      David Jorda Sanchis :  2.9472046641806653
0.6247560266897523 0.5804916921950531
Harold Mayot :  1.3966759408660068      Tristan Lamasine :  3.520949462719722
0.5791758417257415 0.5915321586436059
Hugo Gaston :  2.2943413136680766      Guido Andreozzi :  1.7725937428096665
0.6143212622899322 0.576844978593294
Pedro Cachin :  1.4565666105735036      Renzo Olivo :  3.190260910108774
0.5262606248557214 0.5983637284282601
Timofey Skatov :  5.876169390410905      Elmer Moller :  1.2050790118092538
0.5860098619918268 0.5718728072221966
Timofey Skatov :  1.7434071010520422      Daniel Masur :  2.3451579875748254
0.6296915674487649 0.5389874262425672
Elmer Moller :  1.1342545943265359      Riccardo Bonadio :  8.448534666663154
0.5538624738138843 0.5629839540121693
Timofey Skatov :  2.2132347236738936      Toby Alex Kodat :  1.8242428117881586
0.57500211301026 0.5957787624287213
Daniel Masur :  2.544514925902966      Lucas Gerch :  1.6474524675864641
0.5897417127723581 0.5239076058202106
Elmer Moller :  1.2365748166216635      Carlos Taberner :  5.226992603355689
0.5919033587445338 0.5985998283656526
Riccardo Bonadio :  2.1488296545616      Federico Agustin Gomez :  1.8704510682061088
0.595520362963301 0.6059356600957011
Timofey Skatov :  2.239864845219425      Norbert Gombos :  1.806539522316261
0.6335409148954915 0.5464594108802171
Toby Alex Kodat :  1.1479099035449554      Rodrigo Pacheco Mendez :  7.7608725043625135
0.5787356130848356 0.5976113825261248
Daniel Masur :  2.482856565075535      Vilius Gaubas :  1.6743740585247104
0.5726304150738591 0.544583083394633
Lucas Gerch :  1.550186599283316      Daniel Michalski :  2.8175651702579088
0.6124816313635337 0.5166787500988175
Elmer Moller :  1.115009448451331      Tom Gentzsch :  9.694937793942849
0.5807575885647307 0.5552051421063802
Carlos Taberner :  1.5820404426506867      Michael Vrbensky :  2.7180936696526996
0.569831607742574 0.5718589749714335
Federico Agustin Gomez :  2.043585972523312      Andrea Collarini :  1.9582344208614415
0.6118829473170476 0.5816421603505104
Timofey Skatov :  1.5326756962640717      Mohamed Safwat :  2.8773148597045326
0.6223763590485148 0.6228764727025209
Norbert Gombos :  2.010178041416007      Andrea Picchione :  1.9899245073653151
0.592252499554562 0.5882063494519251
Toby Alex Kodat :  1.9193231557710024      Oscar Otte :  2.0877567846763707
0.5214574309491583 0.5977521069361034
Rodrigo Pacheco Mendez :  6.400130655816829      Adolfo Daniel Vallejo :  1.1851807046414395
0.5703529814742356 0.5492860437977163
Vilius Gaubas :  1.6393122631179133      Evan Furness :  2.5641808513464444
0.6176453985768265 0.6047860025504093
Daniel Masur :  1.7684975051125675      Michael Geerts :  2.301240398761636
0.5789221218204179 0.5418615024644691
Lucas Gerch :  1.4527764957712597      Luca Giacomini :  3.208595210527878
0.6055308186671944 0.6095440076961527
Daniel Michalski :  2.08582528708809      August Holmgren :  1.9209584745274704
0.6432295256804248 0.5369880890778812
Elmer Moller :  1.0915997980679875      Marko Topo :  11.917054634310183
0.5494158964036016 0.5838702539921101
Tom Gentzsch :  3.0818175769752583      Felix Gill :  1.480349484536937
0.5432619341260243 0.5310280125984089
Carlos Taberner :  1.76989826273547      Moez Echargui :  2.2988729139964184
0.6513911810233703 0.5624362288560968
Michael Vrbensky :  1.1452718817215621      Liam Gavrielides :  7.883644571470938
0.6137894517831074 0.5714387841693197
Riccardo Bonadio :  1.410464167198855      Hazem Naw :  3.436266256380759
0.5994629929602868 0.5603837681465067
Federico Agustin Gomez :  1.4372362976569395      Luca Potenza :  3.287092826827957
0.5802456450765265 0.5839408470845658
Andrea Collarini :  2.080302694100248      Mateus Alves :  1.925666487236589
0.6076329305014354 0.541969745360815
Daniel Dutra Da Silva :  1.2409255423952772      Luca Wiedenmann :  5.150659951028932
0.6299705046974585 0.5994731389685841
Andrea Picchione :  1.5347255023619593      Tibo Colson :  2.870118398286329
0.637986234564889 0.6213889386568474
Norbert Gombos :  1.7156934077936086      Mateus Alves :  2.3972463475426893
0.5507114230018298 0.5876472761404123
Luca Giacomini :  3.1946152128584675      Oleksii Krutykh :  1.45566074368796
0.4933619218593987 0.5070892206822446
Adrian Oetzbach :  2.343319577538729      Federico Iannaccone :  1.7444244963899287
0.5550354079604751 0.6395219976320936
Luca Wiedenmann :  7.283069708059533      Kirill Kivattsev :  1.1591578713056871
0.5664808032222791 0.592085004583597
Luca Potenza :  2.713852475318809      Frederico Ferreira Silva :  1.583480792192444
0.5774761971737317 0.5787573467547203
Andrea Picchione :  2.027208152972952      Jose Pereira :  1.9735125223702652
0.5648668532490637 0.5946991868680013
Tibo Colson :  2.8749990766839573      Justin Engel :  1.5333335959655814
0.6303028017685304 0.6065336641414003
Mateus Alves :  1.6156665491223763      Gianmarco Ferrari :  2.624255859645916
0.6502004173793281 0.5851466943779806
Norbert Gombos :  1.255981109731732      Henri Laaksonen :  4.9065382638898605
0.6026439628937836 0.5652088551368261
Oleksii Krutykh :  1.4539959530040398      Paulo Andre Saraiva Dos Santos :  3.202662806536299
0.5769454729241696 0.6408821482756831
Joel Schwaerzler :  4.8518615379488      Kamil Majchrzak :  1.2596147317726591
0.7809600963014954 0.6327314261014271
Kamil Majchrzak :  1.0456771655638768      Ryan Seggerman :  22.892777006961143
0.5796402678645312 0.5587461108245759
Joel Schwaerzler :  1.6430495155904479      Gerard Campana Lee :  2.555090200296318
0.6163937050441932 0.5449631638798845
Kamil Majchrzak :  1.2120066541481265      Alexander Weis :  5.716833082518777
0.6711736680996488 0.7058972977003035
Ryan Seggerman :  2.934888062546989      Neil Oberleitner :  1.5168257633899764
0.530269270912936 0.5544991036779032
Gerard Campana Lee :  2.680006853771079      Marco Cecchinato :  1.5952356668994052
0.5660382429958064 0.565550775475956
Joel Schwaerzler :  1.9897669847854274      Andrew Paulson :  2.0103388124395676
0.5575937919411063 0.5388652009937278
Alexander Weis :  1.670707232246906      Nicolas Alvarez Varona :  2.4909634963230456
0.6122292331062349 0.5564411545534734
Kamil Majchrzak :  1.3038326390200339      Jonas Forejtek :  4.291285634174617
0.6279698002532298 0.7118892453872901
Ryan Seggerman :  6.424397976795954      Matej Dodig :  1.1843522551770203
0.5203107691027198 0.5629891985407717
Neil Oberleitner :  3.516232889245088      Maks Kasnikowski :  1.3974194933522297
0.5657502655600969 0.536357663108913
Marco Cecchinato :  1.5333256995384101      Filip Cristian Jianu :  2.875026837944418
0.5256248031710586 0.5714665814342554
Gerard Campana Lee :  3.693209115452979      Giovanni Fonio :  1.3713042534507414
0.5556672514791536 0.5396217670232664
Andrew Paulson :  1.7103265552036286      Mirza Basic :  2.407803203574912
0.5627598722846179 0.5676578982727338
Joel Schwaerzler :  2.108935105257694      Ergi Kirkin :  1.9017660233306624
0.516223919424448 0.5501895453822618
Nicolas Alvarez Varona :  3.0803044008382745      Chun Hsin Tseng :  1.4806988821429414
0.5091063546821146 0.4995062345443005
Alexander Weis :  1.8136415916215174      Johan Nikles :  2.229042382171106
0.5689341109705743 0.518520646532182
Jonas Forejtek :  1.334475039987357      Nikoloz Basilashvili :  3.9897597143206838
0.5855575787123809 0.5290716929774083
Kamil Majchrzak :  1.2934391892783368      Gauthier Onclin :  4.407861105598498
0.5581229462624396 0.5221340526034929
Matej Dodig :  1.4605256137684384      Alexis Gautier :  3.171431881534433
0.5468474833551015 0.5249553612668169
Neil Oberleitner :  1.6254316004751186      Bor Artnak :  2.598895865255825
0.546706113590278 0.5657699062959769
Maks Kasnikowski :  2.4996763168847016      Orlando Luz :  1.6668105568789096
0.5139324894894639 0.5494465198547084
Marco Cecchinato :  3.152982977293375      Alexandar Lazarov :  1.4644718562787482
0.5499786963412991 0.5380797690441737
Gerard Campana Lee :  1.7758853189392534      Jay Clarke :  2.288850266386202
0.5504817322032118 0.5592142100822919
Mirza Basic :  2.2036226022577488      Samuel Vincent Ruggeri :  1.830825208935264
0.5699726555444992 0.5609052493480768
Andrew Paulson :  1.82573574240516      Jiri Vesely :  2.21104119471352
0.5979367884608391 0.5707307806559611
Joel Schwaerzler :  1.5649948730318126      Sergey Fomin :  2.769927565243046
0.5467473387663975 0.5350761036942813
Ergi Kirkin :  1.779463774795015      Luka Mikrut :  2.282933257883578
0.5537082793330756 0.5761414174896082
Sergey Fomin :  2.608622532420227      Gerald Melzer :  1.6216498773615127
0.5472976548591231 0.5349994018305355
Luka Mikrut :  1.7690922042872372      Ryan Nijboer :  2.300234216945104
0.5582149415550753 0.6457106100685183
Neil Oberleitner :  7.702612282475325      Jerome Kym :  1.1491955610523086
0.6663284037751493 0.7129478497233335
Ryan Seggerman :  3.4356603105607504      Constantin Bittoun Kouzmine :  1.4105662828532008
0.5490240444405335 0.55439061311333
Nikoloz Basilashvili :  2.1207622769266425      Marcelo Zormann Da Silva :  1.892249873668306
0.49299990234839974 0.5204321620991624
Sergey Fomin :  2.808178985150513      Juan Carlos Prado Angelo :  1.553042596010903
0.6858800204449413 0.538459684319025
Ryan Nijboer :  1.0312603659330708      George Goldhoff :  32.98938880437359
0.6382077724239391 0.4602462018597644
Jerome Kym :  1.0095093127234556      Berk Bugarikj  :  106.16007087803682
0.5602744647366739 0.6165064252301587
Neil Oberleitner :  4.312701204464151      Alexander Donski :  1.3018684566698662
0.6061425995182869 0.6542182603265991
Ryan Seggerman :  3.6700119360722194      Calum Puttergill :  1.374530160891742
'''
import re
pattern = re.compile(r":\s+([0-9]*\.[0-9]+)")
matches = pattern.findall(text)

print(matches)

['1.5970421678592348', '2.6749235712874655', '2.831371863290445', '1.5460387483529914', '1.3903406578547965', '3.561864822116459', '1.4722426307344907', '3.11755554225309', '1.2904391190388642', '4.443062364702285', '2.07014400563752', '1.9344536760772368', '1.6927289700059238', '2.443566016867244', '1.4929068861333237', '3.0287807456792466', '1.2769747207394448', '4.610437794937676', '6.646213633103876', '1.177109841210573', '1.964980232464765', '2.036290657939994', '1.747180450838908', '2.3383647803917182', '1.6935028599758655', '2.441955120465979', '1.5243878538561895', '2.906985435772974', '1.8001253971248004', '2.249804097699481', '4.8014152299127995', '1.263059923612433', '1.171501858745317', '6.83084059447435', '1.6317054208687387', '2.583016334773212', '1.7069160538460952', '2.414595119971222', '1.6792236017476567', '2.472269216539264', '3.2690723537474766', '1.4407087320721412', '1.4671788809058264', '3.140507717431644', '1.9017954739152787', '2.108898889964874', '1.7004042785

In [ ]:
i = 0
accuracy = 0
while i<len(matches):
  if(matches[i]<=matches[i+1]):
    accuracy +=1
  i+=2
accuracy/=(len(matches)/2)
accuracy
#this is accuracy of model #1

0.5974025974025974

In [ ]:
test_data['accuracy'] = test_data['winner_odds']<test_data['loser_odds']

In [ ]:
test_data['accuracy'].sum()/len(test_data)
#accuracy of bookmakers

0.5915492957746479

In [ ]:
#3 here we take into account type of surface and number of sets
revenue = 0
bet_sum = 0
for ind, row in test_data.iterrows():
  best_of = row['best_of']
  if(row['surface'] == 'Hard' and row['best_of'] == 3):
    train_players_data = train_players_data_HARD_bestof3
  if(row['surface'] == 'Hard' and row['best_of'] == 5):
    train_players_data = train_players_data_HARD_bestof5

  elif(row['surface'] == 'Clay' and row['best_of'] == 3):
    train_players_data = train_players_data_CLAY_bestof3
  elif(row['surface'] == 'Clay' and row['best_of'] == 5):
    train_players_data = train_players_data_CLAY_bestof5

  elif(row['surface'] == 'Grass' and row['best_of'] == 3):
    train_players_data = train_players_data_GRASS_bestof3
  elif(row['surface'] == 'Grass' and row['best_of'] == 5):
    train_players_data = train_players_data_GRASS_bestof5

  elif(row['surface'] == 'Carpet' and row['best_of'] == 3):
    train_players_data = train_players_data_CARPET_bestof3
  elif(row['surface'] == 'Carpet' and row['best_of'] == 5):
    train_players_data = train_players_data_CARPET_bestof5

  if((row['winner_name'] in train_players_data.columns) and (row['loser_name'] in train_players_data.columns) and row['tourney_name'] in list(train_data['tourney_name'])):
    f_ij = combined_prob_calc(train_players_data, train_data, row['tourney_name'], row['winner_name'], row['loser_name'])[0] #prob of first player to win on his serve
    g_ji = combined_prob_calc(train_players_data, train_data, row['tourney_name'], row['loser_name'], row['winner_name'])[0] #prob of second player to win on his serve
    print(f_ij, g_ji)
    A = (match_prob_calc(f_ij, g_ji, 0, 0, best_of) + 1 - match_prob_calc(g_ji, f_ij, 0, 0, best_of))/2
    if(row['winner_odds']>1/A):
      bet_sum+=1
      revenue+=row['winner_odds']
    elif(row['loser_odds']>1/(1-A)):
      bet_sum+=1
    print(row['winner_name'], ': ', 1/A,'    ', row['loser_name'], ': ', 1/(1-A))

print('revenue: ', revenue)
print('bet_sum: ', bet_sum)

0.6580843047895112 0.6294203223988807
Casper Ruud :  1.564520607858262      Tomas Machac :  2.771414517166885
0.6034676810726357 0.6358692159674746
Tomas Machac :  2.9400585809274116      Novak Djokovic :  1.5154483528646683
0.6664020260188388 0.619675575289939
Casper Ruud :  1.3901198049721764      Flavio Cobolli :  3.563315133594206
0.676135897826167 0.6416825115071387
Novak Djokovic :  1.5077295630174286      Tallon Griekspoor :  2.9695524405886795
0.6395505467587986 0.5831086176111807
Tomas Machac :  1.3071128538358188      Alex Michelsen :  4.256132029350344
0.5977096038242216 0.6009940638936349
Flavio Cobolli :  2.0701773988743843      Alexander Shevchenko :  1.9344245178900272
0.6602527334504446 0.6416461474562474
Casper Ruud :  1.6926754999943205      Sebastian Baez :  2.4436774507084476
0.6538244920199467 0.6187056432000397
Novak Djokovic :  1.4926902305784953      Yannick Hanfmann :  3.0296728815301326
0.7288898867435654 0.6584912415974401
Tallon Griekspoor :  1.2566748659635

In [ ]:
#4 for common opponents method
revenue = 0
bet_sum = 0
for ind, row in test_data.iterrows():
  first_serve = sps.bernoulli.rvs(0.5)
  best_of = row['best_of']
  if(first_serve == 0):
    pr = match_odds(row['winner_name'], row['loser_name'], train_data, best_of)
    if(pr=='no mutual opponents'):
      bet_sum, revenue = bet(row, train_players_data, train_data, first_serve, best_of, bet_sum, revenue)
    else:
      if(row['winner_odds']>1/pr):
        bet_sum+=1
        revenue+=row['winner_odds']
      elif(row['loser_odds']>1/(1-pr)):
        bet_sum+=1
      print(row['winner_name'], ': ', 1/pr,'    ', row['loser_name'], ': ', 1/(1-pr))
  else:
    pr = match_odds(row['loser_name'], row['winner_name'], train_data, best_of)
    if(pr=='no mutual opponents'):
      bet_sum, revenue = bet(row, train_players_data, train_data, first_serve, best_of, bet_sum, revenue)
    else:
      if(row['winner_odds']>1/(1-pr)):
        bet_sum+=1
        revenue+=row['winner_odds']
      elif(row['loser_odds']>1/pr):
        bet_sum+=1
      print(row['loser_name'], ': ', 1/pr,'    ', row['winner_name'], ': ', 1/(1-pr))

print('revenue: ', revenue)
print('bet_sum: ', bet_sum)

['Tim Van Rijthoven', 'Marton Fucsovics', 'Tomas Martin Etcheverry', 'Cameron Norrie', 'Jan Lennard Struff', 'Mikhail Kukushkin', 'Richard Gasquet', 'Emil Ruusuvuori', 'Stefanos Tsitsipas', 'Laslo Djere', 'Sebastian Ofner', 'Botic Van De Zandschulp', 'Diego Schwartzman', 'Matteo Arnaldi', 'Ben Shelton', 'Alex Molcan', 'Stan Wawrinka', 'Lukas Klein', 'Roberto Bautista Agut', 'Jordan Thompson', 'Novak Djokovic', 'Lorenzo Musetti', 'Ryan Peniston', 'Zhizhen Zhang', 'Karen Khachanov', 'Hubert Hurkacz', 'Alexander Bublik', 'Marcos Giron', 'Adrian Mannarino', 'Andrey Rublev', 'Alexandre Muller', 'Laurent Lokoli', 'Elias Ymer']
-0.10454541138489293
-0.21999752418690788
0.07164953010201125
-0.14967025555121682
-0.06128183235650486
-0.15273785038490917
0.10061716626505013
-0.08718024858224827
-0.13181507912132395
-0.09810447005568956
-0.04933357167946206
-0.0800932946674826
-0.2222225460416259
0.19763227945640505
-0.27809540238382735
-0.017556045326031278
0.12256125598793954
0.04922031055146614

In [ ]:
#ROI for method #1
print('ROI: ', (revenue-bet_sum)/bet_sum)

ROI:  -0.07595419847328279


In [ ]:
#ROI for method #2
print('ROI: ', (revenue-bet_sum)/bet_sum)

ROI:  0.0035555555555550587
